In [ ]:
from vllm import LLM, SamplingParams
import random
from tqdm import tqdm
import pandas as pd
llm = LLM("./gemma/7b-it/", max_model_len=4096)
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=3000)

In [ ]:
prompts = pd.read_csv("../data_cleaning/clean_prompt.csv")

In [ ]:
prompts.columns = ["rewrite_prompt"]

In [ ]:
ps = list(prompts["rewrite_prompt"])

In [ ]:
df = pd.read_csv("../datasets/gemini_dataset_v0.csv")
df.query("rewrite_prompt == 'Convert this into a sea shanty with a rousing chorus.'")

In [ ]:
df = pd.read_csv("../datasets/train.csv")

In [ ]:
df

In [ ]:
df["rewrite_prompt"][0]

In [ ]:
# prompts["rewrite_prompt"] = prompts["rewrite_prompt"].apply(lambda x: x[:-1] + ":" if x[-1] == "." else x + ":")

In [ ]:
prompts

In [ ]:
source_text = pd.read_parquet("./train-00000-of-00001.parquet")

In [ ]:
source_text.head()

In [ ]:
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{}<end_of_turn>\n<start_of_turn>model\n"


def format_template(idx, text):
    o_p = prompts.loc[idx, "rewrite_prompt"]
    if o_p[-1] == ".":
        p = o_p[:-1] + ":"
    else:
        p = o_p + ":"

    inst = USER_CHAT_TEMPLATE.format(p + '"""' + text + '"""')
    return o_p,  inst

In [ ]:
original_texts = []
rewritten_texts = []
original_prompts = []

acc = 0
for idx in tqdm(range(len(prompts))):
    i = random.randint(0, len(source_text) - 1)
    text = source_text.loc[i, "top_comment"]
    o_p,  inst = format_template(idx, text)
    outputs = llm.generate([inst], sampling_params)
    for output in outputs:
        original_texts.append(text)
        rewritten_texts.append(output.outputs[0].text)
        original_prompts.append(o_p)
        acc += 1
    if acc != 0 and acc % 100 == 0:
        gen_df = pd.DataFrame({
            "rewrite_prompt": original_prompts,
            "original_text": original_texts,
            "rewritten_text": rewritten_texts
        })
        gen_df.to_csv(f"./datasets/human_essays/gen_{acc}.csv", index=False)

In [ ]:
gen_df = pd.DataFrame({
    "rewrite_prompt": original_prompts,
    "original_text": original_texts,
    "rewritten_text": rewritten_texts,
    # "style": styles
})
gen_df.to_csv(f"../datasets/gemini_dataset_v0.csv", index=False)